In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input/"))

# Any results you write to the current directory are saved as output.

#reading the training and test data
train_data = pd.read_csv('input/google-landmarks-dataset/index.csv')
test_data = pd.read_csv('input/google-landmarks-dataset/test.csv')

print("Training data size:",train_data.shape)
print("Test data size:",test_data.shape)

['.DS_Store', 'google-landmarks-dataset']
Training data size: (1098461, 2)
Test data size: (117703, 2)


In [2]:
#Downloading the images 
from IPython.display import Image
from IPython.core.display import HTML 
def display_image(url):
    img_style = "width: 500px; margin: 0px; float: left; border: 1px solid black;"
    #images_list = ''.join([f"<img style='{img_style}' src='{u}' />" for _, u in urls.head(20).iteritems()])
    image=f"<img style='{img_style}' src='{url}' />"
    display(HTML(image))

In [3]:
#Displaying the images
for x in range(28):
    display_image(train_data['url'][x])

In [4]:
from keras.preprocessing import image
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

Using TensorFlow backend.


In [5]:
import sys
# https://gist.github.com/vladignatyev/06860ec2040cb497f0f3
# vladignatyev's progress bar
# remove later
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()  # As suggested by Rom Ruben (see: http://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console/27871113#comment50529068_27871113)


In [6]:
def setter(x, adder):
    try:
        url1 = train_data['url'][x+adder]
        urllib.request.urlretrieve(url1, 'tempfiles/evaluated.jpg')
        return True
    except:
        return False

In [15]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
import urllib.request
import scipy
import pandas as pd

url0 = train_data['url'][0]
urllib.request.urlretrieve(url0, 'tempfiles/evaluated.jpg')

model = Sequential()
model = ResNet50(include_top=False, weights='imagenet')


evaluated_coords = model.predict(path_to_tensor("tempfiles/evaluated.jpg"))

flat = evaluated_coords.flatten()

dataframe = pd.DataFrame([flat])
print(dataframe)

initial_row = np.full((1,1), 0)
index_dataframe = pd.DataFrame(initial_row)

adder = 1

for x in range(100):
    progress(x,100)
    temp = setter(x, adder)
    while temp is not True:
        adder += 1
        temp = setter(x, adder)
    evaluated_coords = model.predict(path_to_tensor("tempfiles/evaluated.jpg"))
    flat = evaluated_coords.flatten()
    dataframe = dataframe.append(pd.DataFrame([flat]))
    
    current_row = np.full((1,1), x+adder)
    index_dataframe = index_dataframe.append(pd.DataFrame(current_row))
    

print(dataframe)
dataframe.to_csv("data/bottleneck_features.csv", sep=',', encoding='utf-8')
index_dataframe.to_csv("data/indices.csv", sep=',', encoding='utf-8')

/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


   0       1       2       3       4       5       6       7       8       \
0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0   

   9       ...  100342  100343  100344  100345  100346  100347  100348  \
0    -0.0  ...    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0   

   100349  100350  100351  
0    -0.0    -0.0    -0.0  

[1 rows x 100352 columns]
    0       1       2       3       4       5       6       7       8       \
0     -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0   
0     -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0   
0     -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0   
0     -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0   
0     -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0   
0     -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0   
0     -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0 

In [ ]:
def is_same_landmark(row1, row2):